# PypeIt Reduction for the J0100 field

This is the real code I use to reduce the J0100 field.

In [ ]:
import os, sys
sys.path.append('/Users/myue/Research/Projects/JWST/dependencies/PypeIt-development-suite/dev_algorithms/jwst/')

from pathlib import Path
import numpy as np
from astropy.io import fits
from matplotlib import pyplot as plt
from astropy.table import Table

import glob

import utils_myue
import importlib
importlib.reload(utils_myue)

In [ ]:
from astropy.table import Table, vstack, hstack

tbl_info = Table.read('../../MASQUERADE_LAE/J0100_photcat_v4_O3CANDIDATES_all_CLUMPS.fits')
tbl_info['redshift'] = tbl_info['z_O3doublet_combined_n']

In [ ]:
def reduce_pointing(prefix, suffix, spec2dir, outdir=None, fix_persistence=True,\
                    bad_bkg_sources = {}, coadd2d_kwargs = {},\
                    ibkg_list_default=[[1,2], [0,2], [0,1], [4,5], [3,5], [3,4]],\
                   coadd2d_command='pypeit_coadd_2dspec', objids=None):

    if outdir is None:
        outdir = prefix

    outdir = os.path.abspath(outdir)
    
    scipath = os.path.join(outdir, 'Science')
    if not os.path.isdir(scipath):
        #msgs.info('Creating directory for Science output: {0}'.format(scipath))
        os.makedirs(scipath)

    coaddpath = outdir + '/Science_coadd/'
    if not os.path.isdir(coaddpath):
        os.system('mkdir -p %s'%coaddpath)

    nscleanfiles_1 = glob.glob(f'{spec2dir}/{prefix}*nrs1*{suffix}')
    scifiles_1 = [f.replace('_nsclean', '') for f in nscleanfiles_1]
    scifiles_1.sort()
    #print(scifiles_1)

    msareducer = utils_myue.MSAreducer(scifiles_1, outdir, from_msaexp=True)
    
    for ii, (islit, isource) in enumerate(msareducer.slits_sources):

        if isource in bad_bkg_sources.keys():
            ibkg_list = bad_bkg_sources[isource]['bkg_indice']
        else:
            ibkg_list = ibkg_list_default

        if objids is not None:
            if not isource in objids:
                continue

        msareducer.reduce_slit(islit, isource, ibkg_list=ibkg_list)
        
        # step 2: coadd slits
        if isource in coadd2d_kwargs.keys():
            msareducer.coadd2d_source(islit, isource, outdir=coaddpath,\
                coadd2d_command=coadd2d_command,
                **coadd2d_kwargs[isource])
        else:
            msareducer.coadd2d_source(islit, isource, outdir=coaddpath,\
                coadd2d_command=coadd2d_command,\
                find_trim_edge='5,5', find_min_max='400,800')
            
        # construct manual extraction for each object
        msareducer.manual_extraction(islit, isource)
    
    return msareducer

In [ ]:
# this dict stores objects that need specific background treatment
bad_bkg_sources_1 = {}

coadd2d_kwargs_1 = {}

bad_bkg_sources_2 = {'4713_11981': {'bkg_indice':[[2],[0,2],[1], [5],[3,5],[4]]},
                   '4713_11743': {'bkg_indice':[[1],[2],[0], [4],[5],[3]]},}

# this dict stores objects that need specific coadd2d and auto extaction treatment
coadd2d_kwargs_2 = {'4713_6733':{'offsets':[0,0], 'stack_indices':[0,3],\
                               'find_trim_edge':'5,5', 'find_min_max':'400,800'},
                 '4713_11743':{'offsets':[0,-5.28973998,0,-5.28973998], 'stack_indices':[0,1,3,4]}}

In [ ]:
reducer_pointing_1 = reduce_pointing(prefix='jw04713001001', suffix='_nsclean.fits',\
                                    spec2dir='../msaexp_pypeit/pypeit/',\
                                    outdir='./data-release-2/jw04713001001_msaexp/',
                                    objids=None)

In [ ]:
reducer_pointing_1.find_islit_by_isource('4713_16184')

In [ ]:
reducer_pointing_1.coadd2d_source('30', '4713_16184', outdir='./data-release-2/jw04713001001_msaexp/Science_coadd/',\
                coadd2d_command='pypeit_coadd_2dspec',\
                find_trim_edge='5,5', find_min_max='100,800')

In [ ]:
reducer_pointing_2 = reduce_pointing(prefix='jw04713001002', suffix='_nsclean.fits',\
                                    spec2dir='../msaexp_pypeit/pypeit/',\
                                    outdir='./data-release-2/jw04713001002_msaexp/',
                                    bad_bkg_sources=bad_bkg_sources_2,\
                                    coadd2d_kwargs=coadd2d_kwargs_2)

In [ ]:
#reducer_pointing_2.reduce_slit('25', '4713_13864', \
#                               ibkg_list=[[1,2], [0,2], [0,1], [4,5], [3,5], [3,4]])

In [ ]:
#reducer_pointing_1.coadd2d_source('25', '4713_13864', outdir='./data-release-2/jw04713001001_msaexp/Science_coadd/',
#                find_trim_edge='5,5', find_min_max='400,800')
#                coadd2d_command=coadd2d_command)

In [ ]:
def get_ypos_ra_dec(slit_dm, spec2dfile, ra_src, dec_src):
    center = fits.open(spec2dfile)[-2].data['center']
    
    # get the slit-frame y position
    transformer = slit_dm.meta.wcs.get_transform('world', 'slit_frame')
    
    transformer1 = slit_dm.meta.wcs.get_transform('world', 'detector')
    transformer2 = slit_dm.meta.wcs.get_transform('detector', 'slit_frame')

    result = transformer(ra_src, dec_src, 1)

    dx, dy, w = result
    
    detx, dety = transformer1(ra_src, dec_src, 1)
    dxtest, dytest, _ = transformer2(detx, dety+1)
    #print(detx, dety)
    factor = dytest - dy
    dy_pix = dy / factor
    ypos = center + dy_pix
    
    return np.median(ypos)


In [ ]:
# special treatment for 11743

islit = '7'
isource = '4713_11743'
ra, dec = 15.061265, 27.9884172
ibkg_list = bad_bkg_sources_2[isource]['bkg_indice']
outdir='./data-release-2/jw04713001002_msaexp/Science_coadd/'

reducer_pointing_2.reduce_slit(islit, isource, ibkg_list=ibkg_list)

# step 2: coadd slits

if isource in coadd2d_kwargs_2.keys():
    reducer_pointing_2.coadd2d_source(islit, isource, outdir=outdir,\
            **coadd2d_kwargs_2[isource])
else:
    reducer_pointing_2.coadd2d_source(islit, isource, outdir=outdir)
        
# construct manual extraction for each object

scifile1name = os.path.basename(reducer_pointing_2.scifiles_1[0])[:-5]
scifile2name = os.path.basename(reducer_pointing_2.scifiles_1[-2])[:-5]

reducer_pointing_2.Coadd2dSpecHolder[isource] =\
    outdir+f'/spec2d_{scifile1name}-{scifile2name}-{isource}.fits'

slit_dm = reducer_pointing_2.find_slit_by_islit(islit)
ypos = get_ypos_ra_dec(slit_dm, reducer_pointing_2.Coadd2dSpecHolder[isource], ra, dec)

reducer_pointing_2.manual_extraction(islit, isource,\
                  manual_info={'ypos': ypos, 'xpos': 400.0, 'fwhm':3.0,'boxcar_rad':3.0})


In [ ]:
reducer_pointing_2.slits_sources.append(('7', '4713_11981'))

In [ ]:
# special treatment for 11981

islit = '7'
isource = '4713_11981'
ra, dec = 15.0612418, 27.9882913
ibkg_list = bad_bkg_sources_2[isource]['bkg_indice']
outdir='./data-release-2/jw04713001002_msaexp/Science_coadd/'

reducer_pointing_2.reduce_slit(islit, isource, ibkg_list=ibkg_list)

# step 2: coadd slits

if isource in coadd2d_kwargs_2.keys():
    reducer_pointing_2.coadd2d_source(islit, isource, outdir=outdir,\
            **coadd2d_kwargs_2[isource])
else:
    reducer_pointing_2.coadd2d_source(islit, isource, outdir=outdir)
        
# construct manual extraction for each object

scifile1name = os.path.basename(reducer_pointing_2.scifiles_1[0])[:-5]
scifile2name = os.path.basename(reducer_pointing_2.scifiles_1[-1])[:-5]

reducer_pointing_2.Coadd2dSpecHolder[isource] =\
    outdir+f'/spec2d_{scifile1name}-{scifile2name}-{isource}.fits'

slit_dm = reducer_pointing_2.find_slit_by_islit(islit)
ypos = get_ypos_ra_dec(slit_dm, reducer_pointing_2.Coadd2dSpecHolder[isource], ra, dec)

reducer_pointing_2.manual_extraction(islit, isource,\
                  manual_info={'ypos': ypos, 'xpos': 400.0, 'fwhm':3.0,'boxcar_rad':3.0})


# after getting the two pointing reducers, we can save the info to a master table

In [ ]:

reducer_pointing_1.saveinfo(tbl_info)
reducer_pointing_2.saveinfo(tbl_info)

In [ ]:
reducer_pointing_1.tbl_extraction_info['pointing'] = 1
reducer_pointing_2.tbl_extraction_info['pointing'] = 2

In [ ]:
masterinfo = vstack([reducer_pointing_1.tbl_extraction_info, reducer_pointing_2.tbl_extraction_info])

In [ ]:
masterinfo.write('./data-release-2/masterinfo_J0100_pypeit_msaexp.fits', overwrite=True)

# Post-PypeIt clean up

### rename and coadd 1d specs

In [ ]:
masterinfo = Table.read('./data-release-2/masterinfo_J0100_pypeit_msaexp.fits')

In [ ]:
#masterinfo[masterinfo['isource']=='4713_15437']

In [ ]:
def stack_1dspec(filelist, outfile):

    script_string =\
f'''
[coadd1d]
  coaddfile='{outfile}'
  ex_value = 'OPT'
  flux_value = False

coadd1d read
  filename | obj_id
'''
    for fname in filelist:
        with fits.open(fname) as hdulist:
            header = hdulist[1].header
            objid = header['NAME']
            script_string += f'{fname} | {objid}\n'

    script_string +=\
'''
coadd1d end
'''
    with open('coadd1dscript.txt', 'w') as script:
        script.write(script_string)
    os.system('pypeit_coadd_1dspec coadd1dscript.txt')

In [ ]:
!mkdir ./data-release-2/reduced_spec_msaexp_pypeit

In [ ]:
from collections import Counter

isources_counter = Counter(masterinfo['isource'])

for isource, count in isources_counter.items():
    #if not isource=='4713_15612':
    #    continue
    #else:
    #    print(isource)
    if count == 1:
        #continue
        subtbl = masterinfo[masterinfo['isource']==isource]
        assert len(subtbl)==1

        pointing = subtbl['pointing'][0]
        datadir = f'./data-release-2/jw0471300100{pointing}_msaexp/Science_coadd'
        spec2dfile = datadir + '/' + subtbl['spec2dlist'][0]
        spec1dfile = datadir + '/' + subtbl['spec1dlist'][0]
        manu1dfile = datadir + '/' + subtbl['manual1dlist'][0]

        spec1dlist = [spec1dfile]
        manu1dlist = [manu1dfile]

        os.system(f'cp {spec2dfile} ./data-release-2/reduced_spec_msaexp_pypeit/spec2d_{isource}_pointing{pointing}.fits')
        os.system(f'cp {spec1dfile} ./data-release-2/reduced_spec_msaexp_pypeit/spec1d_{isource}_pointing{pointing}.fits')
        os.system(f'cp {manu1dfile} ./data-release-2/reduced_spec_msaexp_pypeit/manu1d_{isource}_pointing{pointing}.fits')

        if os.path.exists(spec1dfile):
            stack_1dspec(spec1dlist, f'spec1d_{isource}_stack.fits')
            os.system(f'mv spec1d_{isource}_stack.fits ./data-release-2/reduced_spec_msaexp_pypeit/')

        
        stack_1dspec(manu1dlist, f'manu1d_{isource}_stack.fits')
        os.system(f'mv manu1d_{isource}_stack.fits ./data-release-2/reduced_spec_msaexp_pypeit/')
    else:
        print(isource)
        #continue
        subtbl = masterinfo[masterinfo['isource']==isource]
        # perform pypeit stacking
        spec1dlist = []
        manu1dlist = []
        for idx in range(len(subtbl)):
            pointing = subtbl['pointing'][idx]
            direct = f'./data-release-2/jw0471300100{pointing}_msaexp/Science_coadd/'
            
            spec2dfile = os.path.abspath(os.path.join(direct, subtbl['spec2dlist'][idx]))
            spec1dfile = os.path.abspath(os.path.join(direct, subtbl['spec1dlist'][idx]))
            spec1dlist.append(spec1dfile)
            manu1dlist.append(spec1dfile.replace('spec1d', 'manual1d'))

            os.system(f'cp {spec2dfile} ./data-release-2/reduced_spec_msaexp_pypeit/spec2d_{isource}_pointing{pointing}.fits')
            os.system(f'cp {spec1dfile} ./data-release-2/reduced_spec_msaexp_pypeit/spec1d_{isource}_pointing{pointing}.fits')
            os.system(f'cp {manu1dfile} ./data-release-2/reduced_spec_msaexp_pypeit/manu1d_{isource}_pointing{pointing}.fits')

        if os.path.exists(spec1dlist[0]) and os.path.exists(spec1dlist[1]):
            stack_1dspec(spec1dlist, f'spec1d_{isource}_stack.fits')
            os.system(f'mv spec1d_{isource}_stack.fits ./data-release-2/reduced_spec_msaexp_pypeit/')
        else:
            
            stack_1dspec(manu1dlist, f'manu1d_{isource}_stack.fits')
            os.system(f'mv manu1d_{isource}_stack.fits ./data-release-2/reduced_spec_msaexp_pypeit/')

        # copy spec2d 
        

# plot some info

In [ ]:
from astropy.coordinates import SkyCoord
from msaexp import msa
from astropy.wcs import WCS
from astropy.nddata import Cutout2D

In [ ]:
masterF115Wimage = '../stack_F115W_pipe4_v4_pipeup_1.8.2_pub0988_20221028.fits'
os.path.exists(masterF115Wimage)

In [ ]:
msafile1 = os.path.abspath('../pre-reduction/ratefiles_per/jw04713001001_01_msa.fits')
msafile2 = os.path.abspath('../pre-reduction/ratefiles_per/jw04713001002_01_msa.fits')

In [ ]:
def plot_infotable(tbl_extraction_info, masterimage, msafilelist, outdir, plotdir):
    for index in range(len(tbl_extraction_info)):
    
        isource=tbl_extraction_info['isource'][index]
        pointing=tbl_extraction_info['pointing'][index]
        msafile = msafilelist[pointing-1]
    
        if int(isource.split('_')[-1])<0:
            continue

        #if not isource=='4713_11743':
        #    continue
            
        spec2dfile=f'{outdir}/spec2d_{isource}.fits'
        spec1dfile=f'{outdir}/spec1d_{isource}.fits'
        manu1dfile=f'{outdir}/manu1d_{isource}.fits'
        if not os.path.exists(spec2dfile):
            #continue
            spec2dfile=f'{outdir}/spec2d_{isource}_pointing{pointing}.fits'
            spec1dfile=f'{outdir}/spec1d_{isource}_pointing{pointing}.fits'
            manu1dfile=f'{outdir}/manu1d_{isource}_pointing{pointing}.fits'

        if not os.path.exists(spec1dfile):
            spec1dfile = manu1dfile

        
        if isource=='4713_11743':
            spec1dfile = manu1dfile
        source_id = tbl_extraction_info['isource'][index]
        imagefile = '../nircam_images/F115W/cutout_%s.fits'%source_id
        
        plt.close('all')
        plotter = utils_myue.MSAPlotter(ra=tbl_extraction_info['RA'][index],\
                             dec=tbl_extraction_info['Dec'][index],\
                             isource=tbl_extraction_info['isource'][index],\
                             redshift=tbl_extraction_info['redshift'][index],\
                             mag=tbl_extraction_info['F115W'][index], \
                             imagefile=imagefile, \
                             spec2dfile=spec2dfile,\
                             spec1dfile=spec1dfile, 
                            msafile = msafile,
                                       zqso=6.327)
    
        plotter.make_cutout(masterimage, imagefile, overwrite=False)
        
        plotter.plot(output=f'{plotdir}/plot_%s_pointing%d.pdf'%(plotter.isource, pointing))
        plt.show()

In [ ]:

plotdir = './data-release-2/plots_msaexp_pypeit/'
os.system(f'mkdir -p {plotdir}')

In [ ]:
plot_infotable(masterinfo, masterF115Wimage, [msafile1, msafile2], \
               outdir='./data-release-2/reduced_spec_msaexp_pypeit/', plotdir=plotdir)

In [ ]:
for index in range(len(tbl_extraction_info)):
    #print(index)
    #try:
    
    isource=tbl_extraction_info['isource'][index]

    if int(isource.split('_')[-1])<0:
        continue
        
    spec2dfile=tbl_extraction_info['spec2dlist'][index]
    
    spec1dfile = tbl_extraction_info['spec1dlist'][index]
    if not os.path.exists(outdir+'/'+spec1dfile):
        
        spec1dfile = tbl_extraction_info['manual1dlist'][index]

    source_id = tbl_extraction_info['isource'][index]
    imagefile = '../nircam_images/F115W/cutout_%s.fits'%source_id
    
    print(source_id)
    
    plt.close('all')
    plotter = utils_myue.MSAPlotter(ra=tbl_extraction_info['RA'][index],\
                         dec=tbl_extraction_info['Dec'][index],\
                         isource=tbl_extraction_info['isource'][index],\
                         redshift=tbl_extraction_info['z_'][index],\
                         mag=tbl_extraction_info['F115W'][index], \
                         imagefile=imagefile, \
                         spec2dfile=outdir+'/'+spec2dfile,\
                         spec1dfile=outdir+'/'+spec1dfile, 
                        msafile = f'./ratefiles/jw0{proj_id}00100{pointing}_01_msa.fits',
                                   zqso=6.327)

    filename = masterimages[0]
    plotter.make_cutout(filename, imagefile, overwrite=False)

    plotter.plot(output=f'./{file_prefix}/plots_v2/plot_%s.pdf'%plotter.isource,\
                    estimated_flux=mag_to_flux(plotter.mag))
    plt.show()

# Update the table: add a column indicating if I need to use manual subtraction

In [ ]:
need_manual = [5595, 17549, 16184, 17577]

In [ ]:
auto_problematic_id = []

In [ ]:
for index in range(len(masterinfo)):
    isource = masterinfo['isource'][index]
    objid = int(isource.split('_')[-1])
    #print(isource)
    spec1dfile = './data-release-2/reduced_spec_msaexp_pypeit/spec1d_%s_stack.fits'%isource

    if not os.path.exists(spec1dfile):
        auto_problematic_id.append(True)
    elif objid in need_manual:
        auto_problematic_id.append(True)
        os.system('rm %s'%spec1dfile)
    else:
        auto_problematic_id.append(False)

In [ ]:
masterinfo['MANUAL'] = auto_problematic_id

In [ ]:
masterinfo.write('./data-release-2/masterinfo_J0100_pypeit_msaexp_update.fits')